In [ ]:
!pip3 install gym

In [ ]:
import gym
import numpy as np

In [ ]:
env = gym.make('FrozenLake-v1',new_step_api=True, render_mode='human')

In [ ]:
n_states = env.observation_space.n
n_actions = env.action_space.n

print(n_states)
print(n_actions)

16
4


In [ ]:
V = np.zeros(n_states)
returns = {}

n_episodes = 100
max_steps = 100

for i in range(n_episodes):
    episode = []
    state = env.reset() # Initialize all the things to 0
    for j in range(max_steps):
        env.render()
        action = env.action_space.sample() #choose the random action
        # print(env.step(action))
        next_state, reward, done, info, prob = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break

    states, actions, rewards = zip(*episode)
    discounts = np.array([0.9**i for i in range(len(rewards))])
    for t in range(len(episode)):
        G = sum(rewards[t:] * discounts[:len(rewards)-t])
        action, state = actions[t], states[t]
        if (state, action) not in returns:
            returns[(state, action)] = []
        returns[(state, action)].append(G)
    #update the val fn
    for state in range(n_states):
        if (state, action) in returns:
            V[state] = np.mean(returns[(state, action)])
print(V)

[0.         0.01304446 0.         0.         0.02391485 0.
 0.         0.         0.02172535 0.         0.695847   0.
 0.         0.34867844 0.8645     0.        ]
